In [17]:
# general
import os
import random
import mmcv
import numpy as np
import cv2
from tqdm import tqdm
import numpy as np

os.environ["CUDA_VISIBLE_DEVICES"] = "2"
import torch
import matplotlib.pyplot as plt
import ssl
ssl._create_default_https_context = ssl._create_unverified_context

# detection
from mmdet.apis import init_detector, inference_detector
from mmdet.utils import register_all_modules
from mmdet.apis import init_detector, inference_detector

# segmentation
import matplotlib.pyplot as plt
from scipy.optimize import linear_sum_assignment

home_dir = os.path.expanduser('~')
############################# TO DELETE
home_dir = '/usr/users/henrich1'
#############################
raw_data_dir = os.path.join(home_dir, 'repos/DaNuMa2024/data/raw_data')
output_data_dir = os.path.join(home_dir, 'repos/DaNuMa2024/data/output_data')


In [60]:
# Function to visualize tracking and save both individual frames and a video
def visualize_mot(images_dir, tracks_file, output_dir, video_output_path, ids_to_visualize=None, bbox_linewidth=2, id_size=1, fps=10):
    np.random.seed(2)
    
    # Load tracking data from file
    tracks = {}
    with open(tracks_file, 'r') as f:
        for line in f:
            data = [int(i) if i.isdigit() else float(i) for i in line.split(',')]
            frame_id, obj_id, x_min, y_min, x_max, y_max = data # assuming the format is [x_min, y_min, x_max, y_max]
            if frame_id not in tracks:
                tracks[frame_id] = []
            tracks[frame_id].append((obj_id, x_min, y_min, x_max, y_max))
    
    # Get the list of image files (frames)
    frame_files = sorted([f for f in os.listdir(images_dir) if f.endswith('.jpg')])
    
    # Initialize output directory for frames and video
    frames_output_dir = os.path.join(output_dir, 'frames')
    os.makedirs(frames_output_dir, exist_ok=True)
    
    # Initialize the video writer
    first_frame = cv2.imread(os.path.join(images_dir, frame_files[0]))
    height, width, _ = first_frame.shape
    fourcc = cv2.VideoWriter_fourcc(*'mp4v')  # Codec for .mp4
    video_writer = cv2.VideoWriter(video_output_path, fourcc, fps, (width, height))
    
    # Initialize a dictionary to store bbox colors for each obj_id
    colors = {}
    
    # Iterate over frames and draw bounding boxes
    for frame_idx, frame_file in enumerate(tqdm(frame_files)):
        frame_id = frame_idx + 1  # Assuming frame ID is based on the order of files
        frame_path = os.path.join(images_dir, frame_file)
        frame = cv2.imread(frame_path)
        
        # If the frame contains tracking data, draw the bounding boxes
        if frame_id in tracks:
            for obj_id, x_min, y_min, x_max, y_max in tracks[frame_id]:
                if ids_to_visualize is not None and obj_id not in ids_to_visualize:
                    continue
                
                # Assign a random color to the object if it's not already in the dictionary
                if obj_id not in colors:
                    colors[obj_id] = tuple(map(int, np.random.choice(range(256), size=3)))
                
                color = colors[obj_id]
                
                # Cast x_min, y_min, x_max, y_max to integers
                x_min, y_min, x_max, y_max = int(x_min), int(y_min), int(x_max), int(y_max)
                
                # Draw bounding box
                cv2.rectangle(frame, (x_min, y_min), (x_max, y_max), color, bbox_linewidth)
                
                # Draw object ID label
                cv2.putText(frame, str(obj_id), (x_min, y_min - 5), cv2.FONT_HERSHEY_SIMPLEX, id_size, color, 2)
        
        # Save the output frame with bounding boxes
        output_frame_path = os.path.join(frames_output_dir, frame_file)
        cv2.imwrite(output_frame_path, frame)
        
        # Write the frame to the video
        video_writer.write(frame)
    
    # Release the video writer
    video_writer.release()

init detector

In [61]:
# 1. initialize the model
config_path = os.path.join(home_dir, 'repos/PigDetect/configs/co-detr/co_dino_swin.py')
checkpoint_codino_path = os.path.join(raw_data_dir, '7_segmentation/pretrained/codino_swin.pth')
register_all_modules(init_default_scope=False)
model = init_detector(config_path, checkpoint_codino_path, device='cuda:0') # cuda:0 for gpu

# 2. run model inference on image
image_path = os.path.join(raw_data_dir, '7_segmentation/images/danuma_1578.jpg')
image = mmcv.imread(image_path, channel_order='rgb')
result = inference_detector(model, image)

scores = result.pred_instances.scores.cpu().numpy()
bboxes = result.pred_instances.bboxes.cpu().numpy()
bboxes = bboxes[scores > 0.5] # filter boxes by score

/usr/users/henrich1/repos/PigDetect/mmdetection/projects/CO-DETR/codetr/transformer.py:1325: UserWarning: If you want to reduce GPU memory usage,                               please install fairscale by executing the                               following command: pip install fairscale.
  warnings.warn('If you want to reduce GPU memory usage, \


09/22 17:35:48 - mmengine - INFO - 
rpn_conv.weight - torch.Size([256, 256, 3, 3]): 
NormalInit: mean=0, std=0.01, bias=0 
 
09/22 17:35:48 - mmengine - INFO - 
rpn_conv.bias - torch.Size([256]): 
NormalInit: mean=0, std=0.01, bias=0 
 
09/22 17:35:48 - mmengine - INFO - 
rpn_cls.weight - torch.Size([9, 256, 1, 1]): 
NormalInit: mean=0, std=0.01, bias=0 
 
09/22 17:35:48 - mmengine - INFO - 
rpn_cls.bias - torch.Size([9]): 
NormalInit: mean=0, std=0.01, bias=0 
 
09/22 17:35:48 - mmengine - INFO - 
rpn_reg.weight - torch.Size([36, 256, 1, 1]): 
NormalInit: mean=0, std=0.01, bias=0 
 
09/22 17:35:48 - mmengine - INFO - 
rpn_reg.bias - torch.Size([36]): 
NormalInit: mean=0, std=0.01, bias=0 
 
09/22 17:35:49 - mmengine - INFO - 
bbox_head.fc_cls.weight - torch.Size([2, 1024]): 
NormalInit: mean=0, std=0.01, bias=0 
 
09/22 17:35:49 - mmengine - INFO - 
bbox_head.fc_cls.bias - torch.Size([2]): 
NormalInit: mean=0, std=0.01, bias=0 
 
09/22 17:35:49 - mmengine - INFO - 
bbox_head.fc_reg.we

/usr/users/henrich1/repos/PigDetect/mmdetection/mmdet/models/dense_heads/anchor_head.py:108: UserWarning: DeprecationWarning: `num_anchors` is deprecated, for consistency or also use `num_base_priors` instead
  warnings.warn('DeprecationWarning: `num_anchors` is deprecated, '


### tracker

1. calculate all bboxes in frame 1
2. then for following frame also calculate the bboxes
3. calculate iou between all boxes in frame 1 and 2 to get iou matrix
4. perform hungarian matching to associate detections into tracks
5. do this for all frames of the video and save the tracks in an easily human readable format (the same that is required by visualize_tracking_mot)
6. visualize the tracks using visualize_tracking_mot i provided. you need to change visualize_tracking_mot so that it works directly with frames and not with mp4 file.

In [62]:
images_dir = os.path.join(raw_data_dir, '9_tracking/video1/images')

In [63]:
# Helper function to calculate IoU between two bounding boxes
def calculate_iou(bbox1, bbox2):
    """
    Calculate IoU between two bounding boxes.
    Args:
        bbox1, bbox2: Bounding boxes [x_min, y_min, x_max, y_max]
    Returns:
        iou: Intersection over Union
    """
    x_min1, y_min1, x_max1, y_max1 = bbox1
    x_min2, y_min2, x_max2, y_max2 = bbox2
    
    # Calculate intersection
    x_min_inter = max(x_min1, x_min2)
    y_min_inter = max(y_min1, y_min2)
    x_max_inter = min(x_max1, x_max2)
    y_max_inter = min(y_max1, y_max2)
    
    if x_max_inter < x_min_inter or y_max_inter < y_min_inter:
        return 0.0
    
    # Intersection area
    intersection_area = (x_max_inter - x_min_inter) * (y_max_inter - y_min_inter)
    
    # Areas of the bboxes
    bbox1_area = (x_max1 - x_min1) * (y_max1 - y_min1)
    bbox2_area = (x_max2 - x_min2) * (y_max2 - y_min2)
    
    # Union area
    union_area = bbox1_area + bbox2_area - intersection_area
    
    # IoU
    return intersection_area / union_area

In [64]:
def match_bboxes(bboxes_frame1, bboxes_frame2, iou_threshold):
    """
    Match bounding boxes between two frames using IoU and Hungarian Algorithm.
    Args:
        bboxes_frame1: Bounding boxes in frame 1 (list of [x_min, y_min, x_max, y_max])
        bboxes_frame2: Bounding boxes in frame 2 (list of [x_min, y_min, x_max, y_max])
    Returns:
        matches: List of tuples where each tuple is (index_in_frame1, index_in_frame2)
    """
    iou_matrix = np.zeros((len(bboxes_frame1), len(bboxes_frame2)))

    for i, bbox1 in enumerate(bboxes_frame1):
        for j, bbox2 in enumerate(bboxes_frame2):
            iou_matrix[i, j] = calculate_iou(bbox1, bbox2)
    
    # Perform Hungarian matching (maximize IoU by minimizing negative IoU)
    row_ind, col_ind = linear_sum_assignment(-iou_matrix)
    
    # Filter matches based on IoU threshold (e.g., ignore low IoU matches)
    matches = []
    for r, c in zip(row_ind, col_ind):
        if iou_matrix[r, c] > iou_threshold:  # Threshold for valid IoU match
            matches.append((r, c))
    
    return matches

In [66]:
# Tracking function for all frames
def track_objects_in_video(images_dir, model, output_track_path, iou_threshold=0.3):
    """
    Perform object detection on each frame and track objects using IoU-based matching.
    Args:
        images_dir: Directory containing images (frames)
        model: Initialized object detection model
        output_track_path: Path to save the tracking result in MOT format
        iou_threshold: IoU threshold for matching
    """
    frame_files = sorted([f for f in os.listdir(images_dir) if f.endswith('.jpg')])
    
    tracks = []
    next_track_id = 0
    active_tracks = {}
    
    for frame_idx, frame_file in tqdm(enumerate(frame_files)):
        # Load frame
        frame_path = os.path.join(images_dir, frame_file)
        frame = mmcv.imread(frame_path, channel_order='rgb')
        
        # Run detection
        result = inference_detector(model, frame)
        bboxes = result.pred_instances.bboxes.cpu().numpy()
        scores = result.pred_instances.scores.cpu().numpy()
        bboxes = bboxes[scores > 0.5]
        
        if frame_idx == 0:
            # Initialize new tracks in the first frame
            for bbox in bboxes:
                tracks.append([frame_idx + 1, next_track_id, *bbox])
                active_tracks[next_track_id] = bbox
                next_track_id += 1
        else:
            # Match current frame bboxes with previous frame tracks
            previous_bboxes = active_tracks.values()
            matches = match_bboxes(previous_bboxes, bboxes, iou_threshold)
            
            # Update existing tracks with matched bboxes
            matched_tracks = set()
            for match in matches:
                track_idx, bbox_idx = match
                track_id = list(active_tracks.keys())[track_idx]
                tracks.append([frame_idx + 1, track_id, *bboxes[bbox_idx]])
                active_tracks[track_id] = bboxes[bbox_idx]
                matched_tracks.add(track_id)

            # Remove inactive tracks
            inactive_tracks = set(active_tracks.keys()) - matched_tracks
            for track_id in inactive_tracks:
                del active_tracks[track_id]
                
            # Start new tracks for unmatched bboxes
            unmatched_bboxes = set(range(len(bboxes))) - {m[1] for m in matches}
            for bbox_idx in unmatched_bboxes:
                tracks.append([frame_idx + 1, next_track_id, *bboxes[bbox_idx]])
                active_tracks[next_track_id] = bboxes[bbox_idx]
                next_track_id += 1
            
    # Save tracks to file
    with open(output_track_path, 'w') as f:
        for track in tracks:
            f.write(','.join(map(str, track)) + '\n')

In [69]:
output_dir_tracking = os.path.join(output_data_dir, '9_tracking/video1')
os.makedirs(output_dir_tracking, exist_ok=True)
tracking_results_path = os.path.join(output_dir_tracking, 'tracking_results.txt')
track_objects_in_video(images_dir, model, tracking_results_path)

151it [00:40,  3.71it/s]


In [70]:

# Example usage:
visualize_mot(
    images_dir=images_dir, 
    tracks_file=tracking_results_path, 
    output_dir=output_dir_tracking,
    video_output_path=os.path.join(output_dir_tracking, 'tracking_results.mp4')
)

  0%|          | 0/151 [00:00<?, ?it/s]

100%|██████████| 151/151 [00:05<00:00, 29.44it/s]
